# Data Cleaning


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import sqlite3

df = pd.read_sql("select * from modules",
                 sqlite3.connect("/content/drive/My Drive/APS360_Style_Transfer/bam_tier_1.sqlite"),
                 index_col="mid")
df1 = pd.read_sql("select * from scores",
                 sqlite3.connect("/content/drive/My Drive/APS360_Style_Transfer/bam_tier_1.sqlite"),
                 index_col="mid")

In [4]:
print(df1.iloc[0,:])

content_bicycle      -8.194265
content_cat         -13.083484
content_tree         -0.140162
emotion_scary       -10.125443
media_oilpaint       -8.102950
content_bird         -6.825498
content_dog          -5.438437
emotion_gloomy       -5.251916
media_3d_graphics   -13.048631
media_pen_ink       -10.792016
content_building     -1.033735
content_flower       -6.437519
emotion_happy        -0.784109
media_comic         -13.372336
media_vectorart     -10.279009
content_cars         -4.741513
content_people       -0.372203
emotion_peaceful     -0.295248
media_graphite      -11.520514
media_watercolor    -11.912029
Name: 489, dtype: float64


In [0]:
df1.sort_values(by=['media_watercolor'], inplace=True, ascending=False)
watercolour_url = df.loc[df1.index[:5000],'src']

df1.sort_values(by=['media_oilpaint'], inplace=True, ascending=False)
oilpaint_url = df.loc[df1.index[:5000],'src']

df1.sort_values(by=['media_pen_ink'], inplace=True, ascending=False)
pen_ink_url = df.loc[df1.index[:5000],'src']

df1.sort_values(by=['media_graphite'], inplace=True, ascending=False)
graphite_url = df.loc[df1.index[:5000],'src']

In [9]:
graphite_url.values

array(['https://mir-s3-cdn-cf.behance.net/project_modules/disp/0d9e1e15659879.56295661790f6.JPG',
       'https://mir-s3-cdn-cf.behance.net/project_modules/disp/916a1421142783.56043b5ac42b6.JPG',
       'https://mir-s3-cdn-cf.behance.net/project_modules/disp/36aa5a15659879.5629562b4685c.JPG',
       ...,
       'https://mir-s3-cdn-cf.behance.net/project_modules/disp/4bc1e818239489.562c63303269e.JPG',
       'https://mir-s3-cdn-cf.behance.net/project_modules/disp/b58a634233464.5601db17e55af.jpg',
       'https://mir-s3-cdn-cf.behance.net/project_modules/disp/feb2e314146297.5634c988cc108.png'],
      dtype=object)

In [0]:
import urllib.request
import random


def downloader(image_url, file_name):
    full_file_name ='/content/drive/My Drive/APS360_Style_Transfer/Style_images/media_graphite/' + str(file_name) + '.jpg'
    try:
      urllib.request.urlretrieve(image_url,full_file_name)
    except:
      return 1

i = 0
num_error = 0

for url in graphite_url.values:
    error = downloader(url, i)
    if error == 1:
      num_error += 1
    else:
      i += 1

In [0]:
def downloader(image_url, file_name):
    full_file_name ='/content/drive/My Drive/APS360_Style_Transfer/Style_images/media_pen_ink/' + str(file_name) + '.jpg'
    try:
      urllib.request.urlretrieve(image_url,full_file_name)
    except:
      return 1

i = 0
num_error = 0


for url in pen_ink_url.values[:1000]:
    error = downloader(url, i)
    if error == 1:
      num_error += 1
    else:
      i += 1

In [0]:
def downloader(image_url, file_name):
    full_file_name ='/content/drive/My Drive/APS360_Style_Transfer/Style_images/media_oilpaint/' + str(file_name) + '.jpg'
    try:
      urllib.request.urlretrieve(image_url,full_file_name)
    except:
      return 1

i = 0
num_error = 0


for url in oilpaint_url.values:
    error = downloader(url, i)
    if error == 1:
      num_error += 1
    elif i == 1000:
      break
    else:
      i += 1  

In [0]:
def downloader(image_url, file_name):
    full_file_name ='/content/drive/My Drive/APS360_Style_Transfer/Style_images/media_watercolor/' + str(file_name) + '.jpg'
    try:
      urllib.request.urlretrieve(image_url,full_file_name)
    except:
      return 1

i = 0
num_error = 0


for url in watercolour_url.values:
    error = downloader(url, i)
    if error == 1:
      num_error += 1
    elif i == 1000:
      break
    else:
      i += 1  

In [0]:
text_file = open('/content/drive/My Drive/APS360_Style_Transfer/fall11_urls.txt', encoding = "ISO-8859-1")

In [0]:
lines = text_file.readlines()

In [0]:
print(lines)